## Import packages and modules

In [56]:
# Import necessary packages and modules

# General packages
import numpy as np
import random
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
from constants.constants import exports_dir

# Import custom modules
from modules.Account import Account
from modules.Dark import DarkToken
from modules.FuelCells import FuelCellsToken
from modules.JourneyPhaseManager import JourneyPhaseManager
from modules.Treasury import Treasury
from modules.LaunchControlCenter import LaunchControlCenter
from modules.EvilAddress import EvilAddress
from modules.Jackpot import Jackpot
from modules.Pool import Pool
from modules.Team import Team
from modules.DAIStableCoin import DAIStableCoin

# Set up initial configurations for the simulation
np.random.seed(42)
random.seed(42)


## Initialize accounts

In [57]:
# Import necessary packages and modules

# Initialize Accounts

# Public account
public_account = Account()

# MagicBox account managed by the team
magic_box_account = Account()

# Team account to collect share of Dark tokens used to mint NFTs
team_account = Team()

# Initialize Dark token
dark_token = DarkToken()

# Mint initial Dark token supply to Public, MagicBox, and Evil address
initial_dark_supply = 1000000  # Total initial supply
public_initial_balance = int(initial_dark_supply * 0.10)
magic_box_initial_balance = int(initial_dark_supply * 0.25)
evil_address_initial_balance = int(initial_dark_supply * 0.65)

dark_token.mint(public_account, public_initial_balance)
dark_token.mint(magic_box_account, magic_box_initial_balance)

# Initialize FuelCells token
fuel_cells_token = FuelCellsToken()

# Initialize JourneyPhaseManager
journey_phase_manager = JourneyPhaseManager()

# Initialize Treasury
treasury_account = Account()
treasury = Treasury(treasury_account, dark_token)

# Initialize Jackpot
jackpot_account = Account()
jackpot = Jackpot(jackpot_account, dark_token, journey_phase_manager, fuel_cells_token)

# Initialize LaunchControlCenter
lcc = LaunchControlCenter(dark_token, fuel_cells_token, treasury_account, jackpot_account, team_account, journey_phase_manager)

# Initialize EvilAddress
evil_address_account = EvilAddress(dark_token, fuel_cells_token, lcc, journey_phase_manager, evil_address_initial_balance)
dark_token.mint(evil_address_account, evil_address_initial_balance)

# Initialize Pool
dai_token = DAIStableCoin()
pool = Pool(dark_token, dai_token)

# Initialize accounts and variables summary
print("Initial Balances:")
print(f"Public Account: {dark_token.balance_of(public_account)} Dark tokens")
print(f"MagicBox Account: {dark_token.balance_of(magic_box_account)} Dark tokens")
print(f"Evil Address Account: {dark_token.balance_of(evil_address_account)} Dark tokens")
print(f"Team Account: {dark_token.balance_of(team_account)} Dark tokens")


Initial Balances:
Public Account: 100000 Dark tokens
MagicBox Account: 250000 Dark tokens
Evil Address Account: 650000 Dark tokens
Team Account: 0 Dark tokens


## Data Storage and Helper Functions

In [58]:
# Initialize data storage
prices = []
volumes = []
nfts_minted = []
evil_address_balances = []
yields_per_journey = []
cumulative_yield_per_journey_1 = []
yields_per_journey_accumulated = []
yield_per_nft_per_journey = []
pool_dark_reserves = []
pool_dai_reserves = []
token_distribution = []

# Helper function to accumulate data
def accumulate_data(journey, phase, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account):
    prices.append({
        'journey': journey,
        'phase': phase,
        'price': pool.get_price()
    })
    volumes.append({
        'journey': journey,
        'phase': phase,
        'volume': pool.total_volume
    })
    evil_address_balances.append({
        'journey': journey,
        'phase': phase,
        'balance': dark_token.balance_of(evil_address_account)
    })
    pool_dark_reserves.append({
        'journey': journey,
        'phase': phase,
        'dark_reserve': pool.dark_reserve
    })
    pool_dai_reserves.append({
        'journey': journey,
        'phase': phase,
        'dai_reserve': pool.dai_reserve
    })
    token_distribution.append({
        'journey': journey,
        'phase': phase,
        'public_balance': dark_token.balance_of(public_account),
        'magic_box_balance': dark_token.balance_of(magic_box_account),
        'evil_address_balance': dark_token.balance_of(evil_address_account),
        'team_balance': dark_token.balance_of(team_account),
        'treasury_balance': dark_token.balance_of(treasury_account),
        'jackpot_balance': dark_token.balance_of(jackpot_account),
        'pool_balance': pool.dark_reserve
    })
    if phase == 3 and journey > 0:
        yield_for_journey = treasury.get_total_yield_for_journey(journey)
        yields_per_journey.append({
            'journey': journey,
            'total_yield': yield_for_journey
        })

        nfts_minted.append({
            'journey': journey,
            'nfts_minted': fuel_cells_token.balance_of(public_account) + fuel_cells_token.balance_of(evil_address_account)
        })

        # Accumulate yield for NFTs minted in journey 1
        cumulative_yield_per_journey_1.append({
            'journey': journey,
            'total_yield': (treasury.get_total_yield_for_journey(1) / journey_phase_manager.get_nft_count(1))
        })

        yield_per_nft_per_journey.append({
            'journey': journey,
            'total_yield': treasury.get_total_yield_for_journey(1)
        })
        # Accumulate yield for NFTs minted in previous journeys
        total_yield_accumulated = treasury.total_yield_distributed
        yields_per_journey_accumulated.append({
            'journey': journey,
            'total_yield': total_yield_accumulated
        })


# Initialize Liquidity Pool

In [59]:
# Initialize Liquidity Pool

# Mint initial DAI supply to MagicBox account
initial_dai_supply = 300000  # Total initial DAI supply for liquidity
dai_token.mint(magic_box_account, initial_dai_supply)

# Define liquidity amounts
dark_liquidity_amount = 100000  # Dark tokens from MagicBox balance
dai_liquidity_amount = 300000   # DAI tokens

# Add liquidity to the pool from MagicBox
pool.add_liquidity(magic_box_account, dark_liquidity_amount, dai_liquidity_amount)

# Print pool status after adding liquidity
print(f"Liquidity Pool Initialized:")
print(pool)
print(pool.get_price())


Liquidity Pool Initialized:
Pool: Dark reserve = 100000, DAI reserve = 300000, Fee = 0.003, Total volume = 0
3.0


## Utility Functions

In [60]:
# Utility functions

# Helper function to simulate trades
def simulate_trades(account, pool, total_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token):
    for _ in range(total_trades):
        trade_type = np.random.choice(["buy", "sell"], p=[buy_pressure, sell_pressure])
        if trade_type == "buy":
            dai_amount = np.random.randint(10, 1000)  # Random DAI amount to buy Dark tokens
            if dai_token.balance_of(account) < dai_amount:
                dai_token.mint(account, dai_amount)  # Mint DAI tokens if account has insufficient balance
            if pool.dai_reserve >= dai_amount:
                pool.buy(account, dai_amount)
                dark_tokens_purchased = dai_amount / pool.get_price()
                nft_purchase_amount = int(dark_tokens_purchased * nft_purchase_share_from_public_buy)
                if journey_phase_manager.get_current_phase() == 1 and dark_tokens_purchased >= nft_purchase_amount:
                    lcc.mint_nft(account, nft_purchase_amount)
        else:
            dark_amount = np.random.randint(10, 1000)  # Random Dark amount to sell for DAI
            if dark_token.balance_of(account) >= dark_amount and pool.dark_reserve >= dark_amount:
                pool.sell(account, dark_amount)


## Variables

In [61]:
# Variables for controlling buy/sell pressures and trades

buy_pressure = 0.6  # 60% of the trades will be buys
sell_pressure = 0.4  # 40% of the trades will be sells
total_trades = 10000  # Total number of trades in the journey
nft_purchase_share_from_public_buy = 0.3  # 30% of the Dark tokens from buys will be used for minting NFTs
public_nft_mint_percentage = 0.10  # 10% of the public Dark token balance will be used to mint NFTs

# Initial pool status
initial_dark_reserve = pool.dark_reserve
initial_dai_reserve = pool.dai_reserve
initial_price = pool.get_price()

print(f"Initial Dark reserve: {initial_dark_reserve}")
print(f"Initial DAI reserve: {initial_dai_reserve}")
print(f"Initial Dark token price: {initial_price}")


Initial Dark reserve: 100000
Initial DAI reserve: 300000
Initial Dark token price: 3.0


## Run through the journeys

In [62]:
# # Journey 1 Simulation

# # Simulate the minting phase
# public_initial_balance = dark_token.balance_of(public_account)
# public_nft_mint_amount = int(public_initial_balance * public_nft_mint_percentage)  # 10% of the public Dark token balance

# evil_address_nft_mint_amount = int(evil_address_account.starting_balance * evil_address_account.b * (evil_address_account.r ** (journey_phase_manager.get_current_journey() - 1)) / 100)

# # Public mints NFTs
# lcc.mint_nft(public_account, public_nft_mint_amount)

# # Evil address mints NFTs
# evil_address_account.mint_fuel_cells()

# # Update phase to lottery phase
# journey_phase_manager.increment_phase()

# # Capture the jackpot balance before the lottery
# initial_jackpot_balance = dark_token.balance_of(jackpot.account)

# # Conduct the lottery
# lottery_result = jackpot.conduct_lottery()
# print(lottery_result)

# # Total lottery payouts
# total_lottery_payouts = sum(jackpot.calculate_payouts(journey_phase_manager.get_current_journey(), initial_jackpot_balance))
# print(f"Total lottery payouts: {total_lottery_payouts}")

# # Public sells all their lottery winnings
# lottery_winnings = jackpot.get_lottery_winnings(journey_phase_manager.get_current_journey(), public_account)
# if lottery_winnings > 0:
#     pool.sell(public_account, lottery_winnings)

# print(f"Total lottery winnings for public: {lottery_winnings}")

# # Simulate trades in Phase 1
# phase_1_trades = int(total_trades * 0.3)  # 30% of the trades in Phase 1
# simulate_trades(public_account, pool, phase_1_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
# print(f"Total volume in Pool at end of Phase 1: {pool.total_volume}")

# # Update phase to yield distribution
# journey_phase_manager.increment_phase()

# # Simulate trades in Phase 2
# phase_2_trades = int(total_trades * 0.5)  # 50% of the trades in Phase 2
# simulate_trades(public_account, pool, phase_2_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
# print(f"Total volume in Pool at end of Phase 2: {pool.total_volume}")

# # Update phase to next journey
# journey_phase_manager.increment_phase()

# # Simulate trades in Phase 3
# phase_3_trades = total_trades - (phase_1_trades + phase_2_trades)  # Remaining trades in Phase 3
# simulate_trades(public_account, pool, phase_3_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
# print(f"Total volume in Pool at end of Phase 3: {pool.total_volume}")

# # Distribute yield to NFT holders
# treasury.distribute_yield(journey_phase_manager)

# # Calculate the total yield given by the treasury
# total_yield_given = treasury.total_yield_distributed
# print(f"Total yield given by the treasury in Journey 1: {total_yield_given}")

# # Calculate the yield per NFT for public and evil address
# public_nft_count = fuel_cells_token.balance_of(public_account)
# evil_nft_count = fuel_cells_token.balance_of(evil_address_account)
# total_nft_count = public_nft_count + evil_nft_count

# yield_per_nft = total_yield_given / total_nft_count if total_nft_count else 0
# print(f"Total yield per NFT: {yield_per_nft}")

# # Final pool status
# final_dark_reserve = pool.dark_reserve
# final_dai_reserve = pool.dai_reserve
# final_price = pool.get_price()

# print(f"Final Dark reserve: {final_dark_reserve}")
# print(f"Final DAI reserve: {final_dai_reserve}")
# print(f"Final Dark token price: {final_price}")

# # Print summary of Journey 1
# print("Journey 1 Summary:")
# print(f"Public Account: {fuel_cells_token.balance_of(public_account)} FuelCells NFTs")
# print(f"Evil Address Account: {fuel_cells_token.balance_of(evil_address_account)} FuelCells NFTs")
# print(f"Treasury Balance: {dark_token.balance_of(treasury_account)} Dark tokens")
# print(f"Jackpot Balance: {dark_token.balance_of(jackpot_account)} Dark tokens")
# print(f"Pool: {pool}")
# print(f"Total trades: {total_trades}")
# print(f"Buy pressure: {buy_pressure * 100}%")
# print(f"Sell pressure: {sell_pressure * 100}%")
# print(f"Total lottery winnings per winner: {lottery_winnings / total_trades if total_trades else 0}")
# print(f"Total volume in Pool: {pool.total_volume}")

# # Move to next journey
# journey_phase_manager.increment_phase()


## Simualte for 33 journeys

In [63]:
# Simulate 33 Journeys with data accumulation

accumulate_data(0, 1, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)
accumulate_data(0, 2, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)
accumulate_data(0, 3, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)


for journey in range(1, 34):
    print(f"Simulating Journey {journey}")

    # Phase 1: Minting Phase
    public_initial_balance = dark_token.balance_of(public_account)
    public_nft_mint_amount = int(public_initial_balance * public_nft_mint_percentage)  # 10% of the public Dark token balance

    # Public mints NFTs
    lcc.mint_nft(public_account, public_nft_mint_amount)

    # Evil address mints NFTs
    evil_address_account.mint_fuel_cells()


    # Simulate trades in Phase 1
    phase_1_trades = int(total_trades * 0.3)  # 30% of the trades in Phase 1
    simulate_trades(public_account, pool, phase_1_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
    print(f"Total volume in Pool at end of Phase 1: {pool.total_volume}")

    # Accumulate data at the end of Phase 1
    accumulate_data(journey, 1, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)
    # Increment to Phase 2
    journey_phase_manager.increment_phase()

    # Phase 2: Lottery Phase
    # Capture the jackpot balance before the lottery
    initial_jackpot_balance = dark_token.balance_of(jackpot.account)

    # Conduct the lottery
    lottery_result = jackpot.conduct_lottery()
    print(lottery_result)

    # Total lottery payouts
    total_lottery_payouts = sum(jackpot.calculate_payouts(journey_phase_manager.get_current_journey(), initial_jackpot_balance))
    print(f"Total lottery payouts: {total_lottery_payouts}")

    # Public sells all their lottery winnings
    lottery_winnings = jackpot.get_lottery_winnings(journey_phase_manager.get_current_journey(), public_account)
    if lottery_winnings > 0:
        pool.sell(public_account, lottery_winnings)

    print(f"Total lottery winnings for public: {lottery_winnings}")

    # Simulate trades in Phase 2
    phase_2_trades = int(total_trades * 0.5)  # 50% of the trades in Phase 2
    simulate_trades(public_account, pool, phase_2_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
    print(f"Total volume in Pool at end of Phase 2: {pool.total_volume}")

    # Accumulate data at the end of Phase 2
    accumulate_data(journey, 2, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)

    # Increment to Phase 3
    journey_phase_manager.increment_phase()

    # Phase 3: Yield Distribution Phase
    # Simulate trades in Phase 3
    phase_3_trades = total_trades - (phase_1_trades + phase_2_trades)  # Remaining trades in Phase 3
    simulate_trades(public_account, pool, phase_3_trades, buy_pressure, sell_pressure, nft_purchase_share_from_public_buy, journey_phase_manager, dai_token)
    print(f"Total volume in Pool at end of Phase 3: {pool.total_volume}")

    # Distribute yield to NFT holders
    treasury.distribute_yield(journey_phase_manager)

    # Accumulate data at the end of Phase 3
    accumulate_data(journey, 3, pool, fuel_cells_token, evil_address_account, treasury, jackpot_account, treasury_account)

    # Increment to next journey (Phase 1 of the new journey)
    if journey < 33:  # Avoid incrementing past the last journey
        journey_phase_manager.increment_phase()


Simulating Journey 1
Total volume in Pool at end of Phase 1: 1737310.1665641838
Lottery conducted and payouts distributed.
Total lottery payouts: 38108.533088999975
Total lottery winnings for public: 24854.503797537003
Total volume in Pool at end of Phase 2: 4860298.021938917
Total volume in Pool at end of Phase 3: 6058847.312352384
Simulating Journey 2
Total volume in Pool at end of Phase 1: 7596559.89412905
Lottery conducted and payouts distributed.
Total lottery payouts: 22258.877373833726
Total lottery winnings for public: 10351.065952320563
Total volume in Pool at end of Phase 2: 10769958.941756224
Total volume in Pool at end of Phase 3: 11956912.08000568
Simulating Journey 3
Total volume in Pool at end of Phase 1: 13565142.783214249
Lottery conducted and payouts distributed.
Total lottery payouts: 15679.31514435483
Total lottery winnings for public: 4845.973141332152
Total volume in Pool at end of Phase 2: 16687679.363059968
Total volume in Pool at end of Phase 3: 17928080.219638

## Graphs

In [64]:
# Generate Graphs

# Convert accumulated data to DataFrames
price_df = pd.DataFrame(prices)
volume_df = pd.DataFrame(volumes)
nfts_minted_df = pd.DataFrame(nfts_minted)
evil_address_balance_df = pd.DataFrame(evil_address_balances)
yields_df = pd.DataFrame(yields_per_journey)
yields_accumulated_df = pd.DataFrame(yields_per_journey_accumulated)
cumulative_yield_df = pd.DataFrame(cumulative_yield_per_journey_1)
yield_per_nft_df = pd.DataFrame(yield_per_nft_per_journey)
pool_dark_reserve_df = pd.DataFrame(pool_dark_reserves)
pool_dai_reserve_df = pd.DataFrame(pool_dai_reserves)
token_distribution_df = pd.DataFrame(token_distribution)

# Plot Price of Dark token across journeys and their phase
fig = go.Figure()
fig.add_trace(go.Scatter(x=price_df['journey'] + price_df['phase'] / 3, y=price_df['price'], mode='lines+markers', name='Price of Dark Token'))
fig.update_layout(title='Price of Dark Token Across Journeys and Their Phase', xaxis_title='Journey and Phase', yaxis_title='Price of Dark Token')
fig.show()

# Plot Cumulative volume of trades across phases and journey
fig = go.Figure()
fig.add_trace(go.Scatter(x=volume_df['journey'] + volume_df['phase'] / 3, y=volume_df['volume'], mode='lines+markers', name='Cumulative Volume'))
fig.update_layout(title='Cumulative Volume of Trades Across Phases and Journey', xaxis_title='Journey and Phase', yaxis_title='Cumulative Volume')
fig.show()

# Plot Total number of NFTs minted in each journey
fig = go.Figure()
fig.add_trace(go.Bar(x=nfts_minted_df['journey'], y=nfts_minted_df['nfts_minted'], name='Total NFTs Minted'))
fig.update_layout(title='Total Number of NFTs Minted in Each Journey', xaxis_title='Journey', yaxis_title='Total NFTs Minted')
fig.show()

# Plot Change of balance of Evil address across each journey and their phase
fig = go.Figure()
fig.add_trace(go.Scatter(x=evil_address_balance_df['journey'] + evil_address_balance_df['phase'] / 3, y=evil_address_balance_df['balance'], mode='lines+markers', name='Evil Address Balance'))
fig.update_layout(title='Change of Balance of Evil Address Across Each Journey and Their Phase', xaxis_title='Journey and Phase', yaxis_title='Balance of Evil Address')
fig.show()

# Plot Total yield generated by NFT minted in each journey
fig = go.Figure()
fig.add_trace(go.Bar(x=yields_accumulated_df['journey'], y=yields_accumulated_df['total_yield'], name='Total Yield Generated'))
fig.update_layout(title='Total Yield Generated by NFTs Minted in Each Journey', xaxis_title='Journey', yaxis_title='Total Yield')
fig.show()

# Plot Cumulative yield per journey for NFTs minted in journey 1
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_yield_df['journey'], y=cumulative_yield_df['total_yield'], mode='lines+markers', name='Cumulative Yield for NFTs Minted in Journey 1'))
fig.update_layout(title='Cumulative Yield for NFTs Minted in Journey 1', xaxis_title='Journey', yaxis_title='Cumulative Yield')
fig.show()

# Plot Change in Dark token reserves in the pool
fig = go.Figure()
fig.add_trace(go.Scatter(x=pool_dark_reserve_df['journey'] + pool_dark_reserve_df['phase'] / 3, y=pool_dark_reserve_df['dark_reserve'], mode='lines+markers', name='Dark Reserve in Pool'))
fig.update_layout(title='Change in Dark Token Reserves in the Pool Across Journeys and Their Phase', xaxis_title='Journey and Phase', yaxis_title='Dark Token Reserve')
fig.show()

# Plot Change in DAI token reserves in the pool
fig = go.Figure()
fig.add_trace(go.Scatter(x=pool_dai_reserve_df['journey'] + pool_dai_reserve_df['phase'] / 3, y=pool_dai_reserve_df['dai_reserve'], mode='lines+markers', name='DAI Reserve in Pool'))
fig.update_layout(title='Change in DAI Token Reserves in the Pool Across Journeys and Their Phase', xaxis_title='Journey and Phase', yaxis_title='DAI Token Reserve')
fig.show()

# Plot Token distribution among the accounts across the journeys in a stacked area chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['magic_box_balance'], mode='lines', stackgroup='one', name='Magic Box Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['pool_balance'], mode='lines', stackgroup='one', name='Pool', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['treasury_balance'], mode='lines', stackgroup='one', name='Treasury Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['jackpot_balance'], mode='lines', stackgroup='one', name='Jackpot Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['evil_address_balance'], mode='lines', stackgroup='one', name='Evil Address Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['public_balance'], mode='lines', stackgroup='one', name='Public Account', hoverinfo='x+y'))
fig.add_trace(go.Scatter(x=token_distribution_df['journey'] + token_distribution_df['phase'] / 3, y=token_distribution_df['team_balance'], mode='lines', stackgroup='one', name='Team Account', hoverinfo='x+y'))
fig.update_layout(title='Token Distribution Among Accounts Across Journeys and Their Phase', xaxis_title='Journey and Phase', yaxis_title='Token Balance')
fig.show()
